# Example Multiprocessing Simple: Process x

This code is the target function for process <b>x</b>. It is in a file called 
<br>
<b>example_multiprocessing_simple_process_x.py</b>.


The code for the target function for process <b>y</b> is in a file called 
<br>
<b>example_multiprocessing_simple_process_y.py</b>

The steps of this target function are as follows:
<ol>
    <li>
        <b><i>Get input queues for processes and specify input queue for THIS process</i></b>. 
        The input queues are obtained from the dict. The code must specify which of these
        queues is the input queue for <b>this</b> process. It does so by the statement:
        <br>
        <code>
            Stream.scheduler.input_queue = q_x
        </code>
    </li>
    <li>
        <b><i>Create the streams in this process.</i></b>. The code must give a name to each
        stream in the process. A stream in a multiprocessing application is specified by
        two parameters: 
        (a) the input queue of the process and (b) the name of the stream. The input queue of
        the process was specified earlier as <b>q_x</b>, and now we create a stream <b>x</b> and
        give it a name <i>'x'</i>.
        <br>
        <code>
            x = Stream(name='x')
        </code>
    </li>
    <li>
        <b><i>Declare the callback functions used in this process</i></b>. The callback function
        prints the new item of the stream and then sends the incremented item to process <b>y</b>.
        A process extends a stream in another process by using the following three parameters:
        (i) The input queue of the receiving process;
        (ii) the name of the receiving stream; and
        (iii) the list of values to extend the receiving stream.
        In this case, the input queue of the receiving process is <b>q_y</b>; 
        the name of the receiving stream is <b>'y'</b>; and
        the list of values to extend the receiving stream is <b>[stream_item + 1]</b>.
        Items on streams to other processes are messages which are 2-tuples consisting of
        (a) the stream name and (b) a <i>list of values</i> that extends the stream. The message
        is serialized by calling <b>json.dumps</b>.
        The serialized message is put on to <b>q_y</b>, the input queue for process <b>y</b>.
        <code>
            receiver_stream_name = 'y'
            print('message received by process x: ', stream_item)
            message = (receiver_stream_name, [stream_item+1])
            json_message = json.dumps(message)
            q_y.put(json_message)
        </code>
    </li>
    <li>
        <b><i>Create agents in this process</i></b>
    </li>
    <li>
        <b><i>Start this process</i></b>
    </li>
</ol>

In [ ]:
import json
import multiprocessing as mp
from stream import Stream
from example_operators import single_item
    
def process_target_x(dict_queues):
    
    #-------------------------------------------
    # 1. SPECIFY INPUT QUEUES FOR THE PROCESSES
    #-------------------------------------------
    q_x = dict_queues['x']
    q_y = dict_queues['y']
    # Specify that the input stream for THIS process
    # is q_x
    Stream.scheduler.input_queue = q_x
    
    #-------------------------------------------
    # 2. SPECIFY STREAMS IN THIS PROCESS
    #-------------------------------------------
    x = Stream(name='x')
    
    #-------------------------------------------
    # 3. SPECIFY CALLBACK FUNCTIONS IN THIS PROCESS
    #-------------------------------------------
    def callback_x(stream_item):
        """
        Send stream_item to the stream called 'y' in
        process_y
        """
        receiver_stream_name = 'y'
        print('message received by process x: ', stream_item)
        message = (receiver_stream_name, [stream_item+1])
        json_message = json.dumps(message)
        # Send message to process_y by putting it process_y's
        # input queue
        q_y.put(json_message)
    
    #-------------------------------------------
    # 4. SPECIFY AGENTS IN THIS PROCESS
    #-------------------------------------------
        
    single_item(in_stream=x, func=callback_x)
    
    #-------------------------------------------
    # 5. START SCHEDULER AND THUS START THIS PROCESS
    #-------------------------------------------
    Stream.scheduler.start()

    return